In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Generic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import signal
from scipy.fft import fftshift
import scipy
from datetime import datetime

#ML Libs
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Lambda
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
from keras import backend as K
import scipy.ndimage as nd
from keras.models import Sequential,load_model
from keras import regularizers
from keras import initializers
from keras import metrics
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Convolution2D, MaxPooling2D, BatchNormalization, ReLU, LeakyReLU, Conv2D, LSTM, LeakyReLU
from keras import layers
from keras.optimizers import adam_v2, adagrad_v2, rmsprop_v2
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, auc, roc_curve, precision_recall_fscore_support, mean_squared_error
#from keras.layers.experimental import preprocessing
from keras.initializers import RandomNormal


Terminal = False

#Loading SQI Matched with clinitcal to the Dataframe

In [ ]:
#Loading SQI Matched with clinical to the Dataframe

id = '1DDf_vluJE-zg--41A0zpP5_9-FU-wsJp' # The shareable link id
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Complete_SQIs_with_Clinical.csv')  
SQI_C = pd.read_csv('Complete_SQIs_with_Clinical.csv')


id = '1zEpiX0yXT16cz3znpo8wjoV_U_8vSoyb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Raw_signals.csv')  
Raw = pd.read_csv('Raw_signals.csv')

if Terminal:
    print("\n SQI with Clinical Match:")
    print(SQI_C)
    print("\n Raw Signals")
    print(Raw)


#Not really need, here for completeness
event = ['event_shock', 'reshock24','diagnosis_admission',\
     'ascites', 'respiratory_distress', 'ventilation_cannula', \
     'ventilation_mechanical', 'ventilation_ncpap', 'bleeding_severe', \
     'cns_abnormal', 'liver_mild', 'pleural_effusion', 'skidney']

event_shock = 'shock_admission'

SQI_C['keep'] = False
for i in range(len(event)):
    event_s = event[i]
    SQI_C['keep'][SQI_C[event_s] == True] = True
    #print("\n Total ", event[i], " events:")

In [ ]:
np.random.seed(7)
#Patient 1 pre shock (Admitted with Shock)
#Patient 2 post shock, in this case P1 and P2 are the same 2009
#Unfortunately Post shock data is much shorter and therefore we split it into smaller windows (Data is still less)
Terminal = False
if Terminal:
  #Print the Study
  print('Study 003-2009 Datetime:')
  print(Raw[(Raw.study_no == '003-2009')].PPG_Datetime)

  #Printing Time of Event
  print('PPG Shock Event Start -2009:')
  print(SQI_C[(SQI_C.study_no == '003-2009') & (SQI_C.keep == True)].PPG_w_s)
  print('PPG Shock Event End -2009:')
  print(SQI_C[(SQI_C.study_no == '003-2009') & (SQI_C.keep == True)].PPG_w_f)

  #Print the Study
  print('Study 003-2103 Datetime:')
  print(Raw[(Raw.study_no == '003-2103')].PPG_Datetime)

  #Printing Time of Event
  print('PPG Shock Event Start -2103:')
  print(SQI_C[(SQI_C.study_no == '003-2103') & (SQI_C.keep == True)].PPG_w_s)
  print('PPG Shock Event End -2103:')
  print(SQI_C[(SQI_C.study_no == '003-2103') & (SQI_C.keep == True)].PPG_w_f)


Raw.PPG_Datetime = pd.to_datetime(Raw.PPG_Datetime)

P1times = datetime.strptime('28072020160400000', '%d%m%Y%H%M%S%f')########## Start Until Shock
P1timef = datetime.strptime('29072020032820200', '%d%m%Y%H%M%S%f') 

P2times = datetime.strptime('29072020033146280', '%d%m%Y%H%M%S%f')########## Shock Until the End
P2timef = datetime.strptime('18092020044800000', '%d%m%Y%H%M%S%f') 

# P3times = datetime.strptime('29072020033146280', '%d%m%Y%H%M%S%f')########## Shock Until the End for Patient 2103
# P3timef = datetime.strptime('18092020044800000', '%d%m%Y%H%M%S%f') 

#Extracting
Patient1 = Raw[(Raw.study_no == '003-2009') & (P1times <= Raw.PPG_Datetime) & (P1timef > Raw.PPG_Datetime)]
Patient2 = Raw[(Raw.study_no == '003-2009') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]
#Patient3 = Raw[(Raw.study_no == '003-2103') & (P2times <= Raw.PPG_Datetime) & (P2timef > Raw.PPG_Datetime)]


#Unnecessary Label Added
Patient1['Label'] = 0 
Patient2['Label'] = 1
#Patient3['Label'] = 1

if Terminal:
  print(Patient1)
  print(Patient2)

#Splititng aerray into windows
Patient1_w = np.array_split(Patient1, 684) #Split into 60 Sec Windows
Patient2_w = np.array_split(Patient2, 77)
#Patient3_w = np.array_split(Patient3, 50)##########

def choose_signal(Patient_w, signal):
  Patient_w_arr = np.empty((len(Patient_w),6000,))
  for i in range(len(Patient_w)):
    Patient_w_arr[i] = Patient_w[i][signal]
  
  return Patient_w_arr

Patient1_w_arr = choose_signal(Patient1_w, 'PLETH_bpf') #Change here for PLETH
Patient2_w_arr = choose_signal(Patient2_w, 'PLETH_bpf')


Patient1_w_arr_l = np.zeros((684,1), dtype=int)
Patient2_w_arr_l = np.ones((77,1), dtype=int)

#Defining Sampling rate
fs = 100

Score_array = np.empty((5,1))

##K-FOLD ML=====================================================================
Data = np.vstack((Patient1_w_arr,Patient2_w_arr))
Labels = np.vstack((Patient1_w_arr_l,Patient2_w_arr_l))




In [ ]:
Data1 = np.vstack((Patient1_w_arr[-77:,:],Patient2_w_arr)) #for a balanced model we use slicing 
Labels1 = np.vstack((Patient1_w_arr_l[-77:,:],Patient2_w_arr_l))
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(Data1, Labels1, stratify=Labels1)

# if Terminal:
#   print(train_index)
#   print(test_index)
#Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

#Training_sets = np.vstack((P1_tr,P2_tr))
Globalmean = np.mean(X_train)
Globalstd = np.std(X_train)


P_train = (X_train - Globalmean) / Globalstd
P_test = (X_test - Globalmean) / Globalstd

#Empty arrays
#P1Sxx = np.empty((len(P1_training),26,32))
#P1Sxxtest = np.empty((len(P1_test),26,32))
PSxx = np.empty((len(P_train),26,26))
PSxxtest = np.empty((len(P_test),26,26))


#Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
#Patient 1

def spectrogram_arr(Input, Output_arr, fs):
  for i in range(len(Input)):
    f, t, Sxx = signal.spectrogram(Input[i], fs)
    fmin = 0 # Hz
    fmax = 10 # Hz
    freq_slice = np.where((f >= fmin) & (f <= fmax))
    # keep only frequencies of interest
    f   = f[freq_slice]
    Sxx = Sxx[freq_slice,:][0]
    Output_arr[i] = Sxx
  return Output_arr

X_train = spectrogram_arr(P_train, PSxx, fs)
X_test = spectrogram_arr(P_test, PSxxtest, fs)

Terminal = False

if Terminal:
  print(X_train.shape)
  print(y_train.shape)
  print(X_test.shape)
  print(y_test.shape)



#MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
#tf.random.set_seed(1234)

#converting to categorical
y_trainf = tf.keras.utils.to_categorical(y_train)
y_testf = tf.keras.utils.to_categorical(y_test)
#X_trainf = X_train.reshape(-1, 26*32)
#X_testf = X_test.reshape(-1, 26*32)

from sklearn.decomposition import PCA


# #print(X_train)
# print(X_test.shape)
# pca = PCA(n_components=26) #No. OF COMPONENTS CHOSEN EMPIRICALLY
# for i in range(X_train.shape[2]):
#   pca = pca.fit(X_train[:,:,i])
#   X_trainf = pca.transform(X_train[:,:,i])
#   X_testf = pca.transform(X_test[:,:,i])


pca = PCA(n_components=80) #No. OF COMPONENTS CHOSEN EMPIRICALLY
X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
pca = pca.fit(X_train_2d)
X_trainf = pca.transform(X_train_2d)
X_testf = pca.transform(X_test_2d)


Terminal = True
print(X_trainf.shape)
print(X_testf.shape)

if Terminal:
  print('Train Labels')
  print(y_trainf.shape)
  print('Train Data')
  print(X_train.shape)

# Simple ANN Start ========================================================================================

model = Sequential()
model.add(Dense(256,'relu', input_shape=(80,)))#,use_bias=True, kernel_initializer='glorot_uniform'))
#model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,'relu'))
model.add(Dropout(0.3))
model.add(Dense(256,'relu'))
model.add(Dropout(0.4))
model.add(Dense(2))
model.add(Activation('Softmax'))

# . . . 
#early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
history = model.fit(X_trainf, y_trainf, epochs=300, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])


# ANN ==========================================================================================

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy and Loss')
plt.ylabel('accuracy/loss')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
plt.show()

score = model.evaluate(X_testf, y_testf)

y_prob= model.predict(X_testf)
y_pred_unseen = y_prob.argmax(axis=-1)

PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

print('\nConfusion Matrix on Unseen Data:')
print(confusion_matrix(y_test, y_pred_unseen))

print('\nReport on Unseen Data:')
print(classification_report(y_test, y_pred_unseen))


  #Sourced from https://www.codegrepper.com/code-examples/python/frameworks/-file-path-python/roc+python+example
  # probs = model.predict_proba(X_testf)
  # preds = probs[:,1]
  # fpr, tpr, threshold = roc_curve(y_test, preds)
  # roc_auc = auc(fpr, tpr)

  # plt.title('Receiver Operating Characteristic on Unseen Data')
  # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  # plt.legend(loc = 'lower right')
  # plt.plot([0, 1], [0, 1],'r--')
  # plt.xlim([0, 1])
  # plt.ylim([0, 1])
  # plt.ylabel('True Positive Rate')
  # plt.xlabel('False Positive Rate')
  # plt.show()

#   Score_array[counter] = score[1]
#   Recall[counter] = PRF[1]
#   Precision[counter] = PRF[0] 
#   F1S[counter] =  PRF[2]
#   counter = counter + 1

# print(Score_array)

# print("\n K-Fold CV Max Test Accuracy:")
# print(np.max(Score_array))

# print("\n K-Fold CV Min Test Accuracy:")
# print(np.min(Score_array))

# print("\n K-Fold CV Average Test Accuracy:")
# print(np.sum(Score_array)/5)
# print("\n K-Fold CV Average Test Recall:")
# print(np.sum(Recall)/5)
# print("\n K-Fold CV Average Test Precision:")
# print(np.sum(Precision)/5)
# print("\n K-Fold CV Average Test F1Score:")
# print(np.sum(F1S)/5)

In [ ]:
# split into train and test sets
train_size = int(len(Data) * 1)
test_size = len(Data) - train_size
Train, Test = Data[0:train_size,:], Data[train_size:len(Data),:]
print(len(Train), len(Test))
labels_tr, labels_te = Labels[0:train_size,:], Labels[train_size:len(Data),:]

#Z-score normalization
Globalmean = np.mean(Train)
Globalstd = np.std(Train)

P_train = (Train - Globalmean) / Globalstd
P_test = (Test - Globalmean) / Globalstd


#Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
  #Patient 1

PSxx = np.empty((len(P_train),26,26))
PSxxtest = np.empty((len(P_test),26,26))

fs = 100

def spectrogram_arr(Input, Output_arr, fs):
  for i in range(len(Input)):
    f, t, Sxx = signal.spectrogram(Input[i], fs)
    fmin = 0 # Hz
    fmax = 10 # Hz
    freq_slice = np.where((f >= fmin) & (f <= fmax))
    # keep only frequencies of interest
    f   = f[freq_slice]
    Sxx = Sxx[freq_slice,:][0]
    Output_arr[i] = Sxx
    return Output_arr

X_train = spectrogram_arr(P_train, PSxx, fs)
X_test = spectrogram_arr(P_test, PSxxtest, fs)


print(X_train.shape)
#print(X_test.shape)


pca = PCA(n_components=80) #No. OF COMPONENTS CHOSEN EMPIRICALLY
X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
#X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
pca = pca.fit(X_train_2d)
X_trainf = pca.transform(X_train_2d)
#X_testf = pca.transform(X_test_2d)

print(X_trainf.shape)
#print(X_testf.shape)

In [ ]:
Data1 = np.vstack((Patient1_w_arr[-77:,:],Patient2_w_arr)) #for a balanced model we use slicing 
Labels1 = np.vstack((Patient1_w_arr_l[-77:,:],Patient2_w_arr_l))
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(Data1, Labels1, stratify=Labels1)

# if Terminal:
#   print(train_index)
#   print(test_index)
#Extracting mean and std from training se to apply onto both the training and test set to avoid data leakage

#Training_sets = np.vstack((P1_tr,P2_tr))
Globalmean = np.mean(X_train)
Globalstd = np.std(X_train)


P_train = (X_train - Globalmean) / Globalstd
P_test = (X_test - Globalmean) / Globalstd

#Empty arrays
#P1Sxx = np.empty((len(P1_training),26,32))
#P1Sxxtest = np.empty((len(P1_test),26,32))
PSxx = np.empty((len(P_train),26,26))
PSxxtest = np.empty((len(P_test),26,26))


#Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
#Patient 1

def spectrogram_arr(Input, Output_arr, fs):
  for i in range(len(Input)):
    f, t, Sxx = signal.spectrogram(Input[i], fs)
    fmin = 0 # Hz
    fmax = 10 # Hz
    freq_slice = np.where((f >= fmin) & (f <= fmax))
    # keep only frequencies of interest
    f   = f[freq_slice]
    Sxx = Sxx[freq_slice,:][0]
    Output_arr[i] = Sxx
  return Output_arr


#converting to categorical
X_train = spectrogram_arr(P_train, PSxx, fs)
X_test = spectrogram_arr(P_test, PSxxtest, fs)

Terminal = True

if Terminal:
  print(X_train.shape)
  print(y_train.shape)
  print(X_test.shape)
  print(y_test.shape)



#MACHINE LEARNING MODEL ==================CUSTOM CNN=========================
#tf.random.set_seed(1234)

#converting to categorical
y_trainf = tf.keras.utils.to_categorical(y_train)
y_testf = tf.keras.utils.to_categorical(y_test)
X_trainf = X_train.reshape(-1, 26*26)
X_testf = X_test.reshape(-1, 26*26)

if Terminal:
  print('Train Labels')
  print(y_trainf.shape)
  print('Train Data')
  print(X_train.shape)

# Simple ANN Start ========================================================================================

model = Sequential()
model.add(Dense(256,'relu', input_shape=(26*26,), kernel_initializer='glorot_uniform'))
#model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,'relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.4))
model.add(Dense(128,'relu', kernel_initializer='glorot_uniform'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('Softmax'))

# . . . 
#early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=100, restore_best_weights=True)
model.compile(optimizer=adam_v2.Adam(learning_rate=3e-4),loss='categorical_crossentropy', metrics=['categorical_accuracy'])
history = model.fit(X_trainf, y_trainf, epochs=150, validation_split=0.1, verbose = 0)#, callbacks=[early_stopping])

# ANN ==========================================================================================

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Accuracy and Loss')
plt.ylabel('accuracy/loss')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'validation_accuracy', 'train_loss', 'validation_loss'], loc='best')
plt.show()

score = model.evaluate(X_testf, y_testf)

y_prob= model.predict(X_testf)
y_pred_unseen = y_prob.argmax(axis=-1)

PRF = precision_recall_fscore_support(y_test, y_pred_unseen, average='macro')

print('\nConfusion Matrix on Unseen Data:')
print(confusion_matrix(y_test, y_pred_unseen))

print('\nReport on Unseen Data:')
print(classification_report(y_test, y_pred_unseen))

In [ ]:
# split into train and test sets
train_size = int(len(Data) * 1)
test_size = len(Data) - train_size
Train, Test = Data[0:train_size,:], Data[train_size:len(Data),:]
print(len(Train), len(Test))
labels_tr, labels_te = Labels[0:train_size,:], Labels[train_size:len(Data),:]

#Z-score normalization
Globalmean = np.mean(Train)
Globalstd = np.std(Train)

P_train = (Train - Globalmean) / Globalstd
#P_test = (Test - Globalmean) / Globalstd


#Calculating spectrograms for each Patient separately on each window (120 windows) within 0 and 20 Hz
  #Patient 1

PSxx = np.empty((len(P_train),26,26))
PSxxtest = np.empty((len(P_test),26,26))

fs = 100

def spectrogram_arr(Input, Output_arr, fs):
  for i in range(len(Input)):
    f, t, Sxx = signal.spectrogram(Input[i], fs)
    fmin = 0 # Hz
    fmax = 10 # Hz
    freq_slice = np.where((f >= fmin) & (f <= fmax))
    # keep only frequencies of interest
    f   = f[freq_slice]
    Sxx = Sxx[freq_slice,:][0]
    Output_arr[i] = Sxx
    return Output_arr

X_train = spectrogram_arr(P_train, PSxx, fs)
X_test = spectrogram_arr(P_test, PSxxtest, fs)


print(X_train.shape)
#print(X_test.shape)
X_trainf = X_train.reshape(-1, 26*26)

# pca = PCA(n_components=80) #No. OF COMPONENTS CHOSEN EMPIRICALLY
# X_train_2d = np.array([features_2d.flatten() for features_2d in X_train])
# #X_test_2d = np.array([features_2d.flatten() for features_2d in X_test])
# pca = pca.fit(X_train_2d)
# X_trainf = pca.transform(X_train_2d)
# #X_testf = pca.transform(X_test_2d)

print(X_trainf.shape)
#print(X_testf.shape)

In [ ]:
def LSTM_test(window, batch_sz,lstm_size,lstm_size2,lstm_size3, Epo, Drop):
  print("WINDOW=======================================================", window)
  print("Batch=======================================================", batch_sz)
  print("Lstm1=======================================================", lstm_size)
  print("Lstm2=======================================================", lstm_size2)
  print("Lstm3========================================================", lstm_size3)
  print("Epochs=======================================================", Epo)
  print("Dropout======================================================", Drop)
  # convert an array of values into a dataset matrix
  def create_dataset(dataset, window=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-window-1):
      a = dataset[i:(i+window),:]
      dataX.append(a)
      dataY.append(dataset[i + window,:])
    return np.array(dataX), np.array(dataY)
  


  #window = 26
  train_X, train_Y = create_dataset(X_trainf, window)
  #test_X, test_Y = create_dataset(X_test, window) ########################

  print(train_X.shape)
  print(train_Y.shape)
  #print(test_X.shape)
  #print(test_Y.shape)

  train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 80))
  #train_Y = np.reshape(train_Y, (train_Y.shape[0], train_Y.shape[1], 80))
  #test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 80))
  #batch_size = 32
  acti = LeakyReLU(alpha=0.3)
  return_seq = False
  if lstm_size2 != 0 or lstm_size3 != 0:
    return_seq = True

  rnn = Sequential()    
  rnn.add(LSTM(lstm_size,acti,return_sequences=return_seq, input_shape = train_X.shape[1:], kernel_initializer=RandomNormal(mean=0.0, stddev=0.1, seed=None)))
  if lstm_size2 != 0:
    rnn.add(Dropout(Drop))
    rnn.add(LSTM(lstm_size2,acti, return_sequences=return_seq))
  if lstm_size3 != 0:
    rnn.add(Dropout(Drop))
    rnn.add(LSTM(lstm_size3,acti))
  rnn.add(Flatten())
  rnn.add(Dense(80))
  rnn.compile(loss = "mean_squared_error",  optimizer = "rmsprop", metrics = ['mse'])

  rnn.fit(train_X, train_Y, epochs=Epo, batch_size=batch_sz, verbose=1)


  test1 = rnn.predict(train_X)
  #test2 = rnn.predict(test_X)
  print(test1.shape)
  #print(test2.shape)


  test1 = rnn.predict(train_X)
  #test2 = rnn.predict(test_X)
  print('Test train and Test test shapes:')
  print(test1.shape)
  #print(test2.shape)
        

  predictions1 = model.predict(test1)
  #predictions2 = model.predict(test2)

  predictions_tr = np.empty((len(predictions1),1))
  #predictions_te = np.empty((len(predictions2),1))

  for i in range(len(predictions1)):
    if predictions1[i,0] < predictions1[i,1]:
      predictions_tr[i] = 1
    else:
      predictions_tr[i] = 0

  # for j in range(len(predictions2)):
  #   if predictions2[j,0] < predictions2[j,1]:
  #     predictions_te[j] = 1
  #   else:
  #     predictions_te[j] = 0

  rnn.summary()
  cls1 = classification_report(labels_tr[-test1.shape[0]:],predictions_tr)#,output_dict=True)
  #cls2 = classification_report(labels_te[-test2.shape[0]:],predictions_te,output_dict=True)

  cm1 = confusion_matrix(labels_tr[-test1.shape[0]:], predictions_tr)
  #cm2 = confusion_matrix(labels_te[-test2.shape[0]:], predictions_te)
  
  print(cm1)
  print(cls1)

  #vars = np.array((window, batch_sz,lstm_size,lstm_size2,lstm_size3, Epo, Drop,acti))

  #var = pd.DataFrame(vars).transpose()
  #cls = pd.DataFrame(cls1).transpose()
  #clst = pd.DataFrame(cls2).transpose()

  #Results = pd.concat((cls,clst), axis = 1)

  return



In [ ]:
LSTM_test(32, 80, 32, 288, 0, 300, 0)

In [ ]:
# acti = ['sigmoid']
# for i in range(1):
#   Window_tests, var = LSTM_test(5, 32,32, 0, 0, 500, 0.4, acti[i])
#   if i == 0:
#     Results = Window_tests
#     Results2 = var
#   else:
#     Results = pd.concat((Results,Window_tests), axis = 0)
#     Results2 = pd.concat((Results2,var), axis = 0)

# print(Results)
# print(Results2)

In [ ]:
for i in range(28):
  Window_tests, var = LSTM_test(i+1, 32,32, 0, 0, 500, 0.2)
  if i == 0:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsWindow.csv') 
files.download('ResultsWindow.csv')

Results2.to_csv('VarsWindow.csv') 
files.download('VarsWindow.csv')

In [ ]:
for i in range(16,100,16):
  Window_tests, var = LSTM_test(9, i,32 ,0,0, 500, 0.2)
  if i == 16:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsBatch.csv') 
files.download('ResultsBatch.csv')

Results2.to_csv('VarsBatch.csv') 
files.download('VarsBatch.csv')

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
for i in range(32,300,32):
  Window_tests, var = LSTM_test(9, 32, i ,0 ,0 , 500, 0.2)
  if i == 32:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsLSTM.csv') 
files.download('ResultsLSTM.csv')

Results2.to_csv('VarsLSTM.csv') 
files.download('VarsLSTM.csv')

# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
for i in range(32,300,32):
  Window_tests, var = LSTM_test(9, 32, 32 , i ,0 , 250, 0.2)
  if i == 32:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsLSTM2.csv') 
files.download('ResultsLSTM2.csv')

Results2.to_csv('VarsLSTM2.csv') 
files.download('VarsLSTM2.csv')
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
for i in range(32,300,32):
  Window_tests, var = LSTM_test(9, 32, 32 , 64 , i, 250, 0.2)
  if i == 32:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsLSTM3.csv') 
files.download('ResultsLSTM3.csv')

Results2.to_csv('VarsLSTM3.csv') 
files.download('VarsLSTM3.csv')

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
for i in range(100,1500,100):
  Window_tests, var = LSTM_test(9, 32, 32 , 0, 0, i, 0.2)
  if i == 100:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsEpochs.csv') 
files.download('ResultsEpochs.csv')

Results2.to_csv('VarsEpochs.csv') 
files.download('VarsEpochs.csv')

In [ ]:
for i in np.arange(0,0.6,0.1):
  Window_tests, var = LSTM_test(9, 32, 32 , 0, 0, 250, i)
  if i == 0:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsDropout.csv') 
files.download('ResultsDropout.csv')

Results2.to_csv('VarsDropout.csv') 
files.download('VarsDropout.csv')

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
for i in np.arange(0,0.6,0.1):
  Window_tests, var = LSTM_test(9, 32, 32 , 64, 0, 250, i)
  if i == 0:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsDropout2.csv') 
files.download('ResultsDropout2.csv')

Results2.to_csv('VarsDropout2.csv') 
files.download('VarsDropout2.csv')

In [ ]:
for i in np.arange(0,0.6,0.1):
  Window_tests, var = LSTM_test(9, 32, 32 , 64, 0, 250, i)
  if i == 0:
    Results = Window_tests
    Results2 = var
  else:
    Results = pd.concat((Results,Window_tests), axis = 0)
    Results2 = pd.concat((Results2,var), axis = 0)

print(Results)
print(Results2)

from google.colab import files
Results.to_csv('ResultsDropout3.csv') 
files.download('ResultsDropout3.csv')

Results2.to_csv('VarsDropout3.csv') 
files.download('VarsDropout3.csv')